# Week 5 (Unit 4) lab: Multiprocessing in Python

## Introduction

In this exercise, we are exploring the multiprocessing capabilities of Python to support the concepts of Processes, Process Lifecycle and Concurrent Execution. Over the last few weeks we've examined the time complexity of several different algorithms for sorting and matrix multiplication. To interrogate their behaviour, we instantiated the algorithm `N` times and were evaluating these _sequentially_, calculating the mean running time and standard deviation and examining how the dependent variable (time) changes as we linearly increase some independent variable (size of list or matrix). 

Fundamentally, this approach is going to be quite slow, since it's only leveraging a single available CPU core. For example, running 100 repeats sequentially of a product between two 250x250 square matrixes can take quite a long time. But we might have multiple CPU cores available to us, and we can leverage these by creating multiple worker processes to get results much faster. But we have to be careful, if we do this incorrectly, we will add noise to our results making them much less reliable and/or harder to interpret.

Today's lab provides you with both the tools and the intutition for how to find the critical threshold that maximises use of the resources available to you on any given machine (this approach is scalable beyond a single machine). You are tasked with running a `core_investigation` to find the critical threshold for the machine you are working on (i.e. that doesn't add noise to the running time of an algorithm). You are provided with an arbitrary algorithm, contained in `defs.py` - the function `work_that_CPU` - and we want to minimise the amount of time it takes to run `N` instantiations of the algorithm, while keeping the amount of work within each instantiation fixed.

You should empirically demonstrate what the optimal number of processes is and _explain why_. Think about what is going on within the machine. Why would results be unreliable when we've breached the critical threshold?

Once you have completed this, you are then tasked with splitting up the resources you do have available in a way that will provide you with the best assessment of the performance of the algorithm in question.

## Exercises

First let's import what we need from the multiprocessing library, and *time*, for timing code.

In [ ]:
from multiprocessing import Pool
import time
import os

The multiprocessing package supports the spawning of separate processes running your Python code. In the following code, we create the given number of processes. Note that these are operating system level processes with their own threads of control and address spaces.

Let's investigate how processes are mapped to cores. For this, we're going to create a worker function that executes a lot of instructions (CPU-Bound), and we're going to time how long it takes. 

This worker function must be placed in an auxiliary file, here *defs.py*. Take a look at this file, and then import it by running the cell below.

In [ ]:
import defs

In defs, there is a function called *work_that_CPU*, which will generate some random numbers (to the amount specified by the input *num_loops*), and save the time taken to do this, as well as the name of the process that is executing it.  

Next we write a function that will run *work_that_CPU* a number of times (as specified by the input *num_processes*), with each run being given to a different process.

In [ ]:
def core_investigation(num_processes,num_loops):
    
    inputs=[num_loops]*num_processes
    pool_of_processes = Pool(processes=num_processes)# Create processes to the amount specified by num_processes
    pool_of_processes.map(defs.work_that_CPU,inputs) # Assign a run of work_that_CPU to each process, with num_loops as the input
        
    pool_of_processes.close()  # It is good practice for sake of efficiency to kill the processes once you're done with them.

Each of our processes will be mapped to a physical core. Starting from 1 process, the time taken for each process should remain roughly constant until we exceed the available number of cores (which could range from 4 to 16 depending on your machine).

To see this, play around with the number of processes (*num_processes*) and the amount of computation in each process (*num_loops*) below. Settle on a value for *num_loops* so that the execution of this for a single process takes around 1s to complete.

In [ ]:
num_loops=10000000
num_processes=1
core_investigation(num_processes,num_loops)

# Get the run-times for each of the processes, print them out, and then delete the files that store these.
for filename in os.listdir('.'):
    if filename.startswith('TimedProcess'):
        with open(filename, 'r') as file:
            content = file.read()
            print(content)
        os.remove(filename)

If each process is mapped to a different core, then the time taken should remain roughly constant. If not, then execution time should increase. 

Now run this for all values of *num_processes* from 1 up to 20, and compute for each run the total amount of time taken to run all the processes. Plot a graph of number of processes executing *work_that_CPU* against total time taken.

Note: you do not now need to save to disk all the times taken for every process here, so comment out the lines that do this in *work_that_CPU* in *defs.py*. To keep track of progress just print every value of *num_processes* as you get to it.

You should notice that the total run-time is the same amount of time that it takes each process to run, even when there are more processes than cores available. Why do you think this is the case?

> _Answer_: The scheduler in your computer makes sure that all processes that you've created take turns at running, so they keep switching in and out of the CPU. The first process doesn't just hog the CPU until it is finished!

Now run *work_that_CPU* 20 times without using multiple processes, and time how long it takes. It should take a lot longer.

Now, you're going to keep fixed at 20 the number of times that you're running work_that_CPU, and vary the number of concurrent processes that will do this job, from 1 up to 20. You'll then compute the run-times and plot them. That is, you'll plot number of concurrently running processes against time taken.

To do this, the first step is to write a modified version of the *core_investigation* function that now contains separate inputs for the number of times *work_that_CPU* will be run (*num_repeats*) and the number of processes that will run concurrently (*num_processes*). 

> Hint: You will do a certain number of rounds with *work_that_CPU* given to *num_processes* processes, until the number of repeats left to do is less than *num_processes*. Then you do one more round with the number of processes being the number of repeats you still need to run.

Now, compute the run-times and make the plot. Keep fixed at 20 the number of times that you're running *work_that_CPU*, and vary the number of concurrent processes that will do this job, from 1 up to 20. Compute the run-times and plot them. That is, plot number of concurrently running processes against time taken.

> Hint: Do not use lots of repeats in the *timeit* function. You don't want this to take too long. Also consider printing progress reports along the way so you know how long you're going to have to wait for output.

**Optional extension.** Repeat the above exercise running some other functions multiple times. Use a function that actually has some output (which will need to be sent back to the parent process, i.e. the rest of the Jupyter notebook). Do you always get speed up by parallelising? Play around with functions that take different lengths of time to run, and which return different quantities of data.